    Copyright (C) 2022 Allen Buskirk

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [1]:
from threading import Thread
import os
import re

import pandas as pd
import numpy as np
from multiprocessing import Pool 
import pickle 

#plotting with Bokeh:
from bokeh.io import output_notebook, push_notebook, export, curdoc
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CDSView, HoverTool, LinearColorMapper, FixedTicker, ColorBar, NumeralTickFormatter
from bokeh.layouts import column, gridplot
from bokeh.transform import linear_cmap
from bokeh.transform import factor_cmap
from bokeh.palettes import brewer
import colorcet as cc
output_notebook()

#adding iPython interaction to bokeh plots
from IPython.display import display
from ipywidgets import interact, IntSlider, ToggleButtons, SelectionSlider

#Reading bam files
import pysam


Loading BokehJS ...

The genotype of the HAP1 cells differs from the hg38 standard annotation. To document the variation and identify SNPs, I made a BCF file using the RNAseq data (untreated samples, three biological replicates).

The bcftools suite has commands to make VCF files (run in unix) using the fasta file and the three RNAseq fastq files listed in bam_list.txt:

bcftools mpileup --output-type b --threads 20 -f /home/allen/annotations/hg38_ensemble/files/Homo_sapiens.GRCh38.dna.primary_assembly.fa -b bam_list.txt | bcftools call --threads 20 --output-type b -mv > snp_rnaseq_unt.bcf

Then I made an index using tabix so that pysam can access the BCF (again running this command in unix):

tabix -p bcf snp_rnaseq_unt.bcf

this saves a second file snp_rnaseq_unt.bcf.csi

In [10]:
from pysam import VariantFile

bcf = VariantFile('/home/allen/code/4suv/github_4suv/snp_rnaseq_unt.bcf', 'r')

bcf_dict = {}
for rec in bcf.fetch():
    bcf_dict[counter] = [rec.chrom, rec.pos, rec.alleles[0], rec.alleles[1]]
    
bcf_df = pd.DataFrame.from_dict(bcf_dict, orient='index', columns=['chrom', 'pos', 'ref', 'alt'])

In [ ]:
bcf_df = bcf_df.rename(columns={"pos": "site"})
bcf_df['pos'] = bcf_df.chrom.astype('str') + '_' + bcf_df.site.astype('str')
pickle.dump(bcf_df, open('/home/allen/code/4suv/github_4suv/bcf_df_pkl', 'wb'))

In [7]:
### get conversion statistics from the sorted bam files
### ignore positions that are SNPs in the untreated RNAseq data (using the bcf_df that was pickled)
### looks for T to C conversions, counting them and their positions in the ribosome footpritns
### currently set to only look at 30 mers 
### can set to if read.query_length < 58: in line 27 to look only at the long mers
### reads > 80 were discarded -- junk from the sequencing


def get_conversion_statistics(sample):
    
    totalreads = 0
    minus_snp = 0 
    plus_snp = 0
    conv = 0

    conv_pos = [0]*100 # the position of the conversion in the reads
    conv_len = [0]*100 # the length of reads that have conversions in them
    un_len= [0]*100    # the length of reads without conversions 
    
    align_path = '/home/allen/data/4suv/genome_aligned/riboseq_oct/'+sample+'/'
    bamfile = pysam.AlignmentFile(align_path+sample+"_Aligned.out.sorted.bam","rb")
    
    for read in bamfile.fetch():  

        has_conversion = False

        if read.query_length != 30:
            continue    

        if read.query_length > 80:
            continue

        totalreads += 1 

        if read.get_tag('NM') >= 1:

            if (('I' in read.cigarstring) or ('D' in read.cigarstring) or ('S' in read.cigarstring)):
                continue

            MD = read.get_tag('MD')

            if read.is_reverse & ('A' in MD):
                mismatches = [x.start() for x in re.finditer('a', read.get_reference_sequence())]
                for i in mismatches:
                    if read.query_alignment_sequence[i] == 'G':
                        pos = read.reference_start + 1 + i
                        if (read.reference_name + '_' + str(pos)) in bcf_df.pos.values:
                            minus_snp += 1
                        else:
                            conv += 1
                            conv_pos[read.query_alignment_length - i] += 1
                            has_conversion = True

            elif (not read.is_reverse) & ('T' in MD):
                mismatches = [x.start() for x in re.finditer('t', read.get_reference_sequence())]
                for i in mismatches:
                    if read.query_alignment_sequence[i] == 'C':
                        pos = read.reference_start + 1 + i
                        if (read.reference_name + '_' + str(pos)) in bcf_df.pos.values:
                            plus_snp += 1
                        else:
                            conv += 1
                            conv_pos[i] += 1
                            has_conversion = True

        if has_conversion:
            conv_len[read.query_length] += 1
        else:
            un_len[read.query_length] += 1                        

    print(sample)
    print(totalreads,'reads analyzed')
#     print(minus_snp, 'minus snp')
#     print(plus_snp, 'plus snp')
    print(conv, 'true conversions')
    
    return [sample, totalreads, conv_pos, conv_len, un_len]

In [8]:
filelist = [
 'RIBO_UNT_R1',
 'RIBO_UNT_R2',
 'RIBO_UNT_R3',
 'RIBO_4SU_R1',
 'RIBO_4SU_R2',
 'RIBO_4SU_R3',
 'RIBO_4SU_UV_R1',
 'RIBO_4SU_UV_R2',
 'RIBO_4SU_UV_R3',
 'RIBO_UV_R1',
 'RIBO_UV_R2',
 'RIBO_UV_R3'
]

In [9]:
outdict = {}

p = Pool(12)
map_out = p.map(get_conversion_statistics, filelist)
p.close()

for entry in map_out:
    outdict[entry[0]] = entry[1:]

### [totalreads, conv_pos, conv_len, un_len]
### keys are the sample names

RIBO_UV_R1
677578 reads analyzed
1592 true conversions
RIBO_4SU_UV_R2
301641 reads analyzed
3247 true conversions
RIBO_UNT_R1
769368 reads analyzed
1624 true conversions
RIBO_4SU_UV_R3
415386 reads analyzed
3690 true conversions
RIBO_4SU_R2
529468 reads analyzed
3013 true conversions
RIBO_4SU_R1
559209 reads analyzed
3559 true conversions
RIBO_UV_R3
1039946 reads analyzed
2470 true conversions
RIBO_UNT_R2
1096483 reads analyzed
2591 true conversions
RIBO_UV_R2
1269577 reads analyzed
2883 true conversions
RIBO_4SU_R3
728106 reads analyzed
4269 true conversions
RIBO_UNT_R3
1336752 reads analyzed
2912 true conversions
RIBO_4SU_UV_R1
884042 reads analyzed
6493 true conversions


In [10]:
# percentage of reads with a conversion 
# == 100 * reads with conversions / total reads

for fn in filelist:
    print(fn, 100 * sum(outdict[fn][2]) / outdict[fn][0])

RIBO_UNT_R1 0.20549333998814612
RIBO_UNT_R2 0.2282753129779486
RIBO_UNT_R3 0.2117071827833435
RIBO_4SU_R1 0.6249899411490158
RIBO_4SU_R2 0.5596183338747573
RIBO_4SU_R3 0.5771137718958503
RIBO_4SU_UV_R1 0.722703219982761
RIBO_4SU_UV_R2 1.0608637419979379
RIBO_4SU_UV_R3 0.8755711555035557
RIBO_UV_R1 0.224623585771675
RIBO_UV_R2 0.22188492702687587
RIBO_UV_R3 0.22770413079140647


In [11]:
x_val = [x for x in range(100)]

p = figure(sizing_mode="stretch_width", height=300, title='conversion position',
           y_axis_label='# conversions (normalized)', x_axis_label='position in the read (nt)')

for fn in filelist:
    if not 'R2' in fn:
        continue
    totalreads = outdict[fn][0] / 1000000
    pos_list = np.array(outdict[fn][1])
    if 'RIBO_UNT_R' in fn:
        c = 'black'
    elif 'RIBO_4SU_R' in fn:
        c = 'blue'
    elif 'RIBO_4SU_UV_R' in fn:
        c = 'red'
    elif 'RIBO_UV_R' in fn:
        c = 'green'
    p.line(x_val, pos_list / totalreads, color = c, line_width = 2)

show(p)

In [13]:
### this takes just replicate 2 for each sample
### and normalizes by the number of total reads in the sample

r2_dict = {}
for fn in filelist:
    if not 'R2' in fn:
        continue
    totalreads = outdict[fn][0] / 1000000
    pos_list = np.array(outdict[fn][1])
    r2_dict[fn] = pos_list / totalreads


In [14]:
r2_df = pd.DataFrame.from_dict(r2_dict)
r2_df.head()

,RIBO_UNT_R2,RIBO_4SU_R2,RIBO_4SU_UV_R2,RIBO_UV_R2
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000
2,69.496655,188.736053,125.529765,43.995274
3,82.132410,161.773759,221.171492,69.135430
4,94.768165,323.547519,245.081923,131.985821


In [16]:
### these normalized data from the rep2 samples were saved as a csv
### and plotted separately for the publication 

r2_df.to_csv('/home/allen/data/4suv/cpm_30_r2.csv')

The following code was used to calculate the C to T conversions for the 30 mers as a control

In [2]:
bcf_df = pickle.load(open('/home/allen/code/4suv/github_4suv/bcf_df_pkl', 'rb'))

In [3]:
## for C to T conversions

def get_other_stats(sample):
    
    totalreads = 0
    minus_snp = 0
    plus_snp = 0
    conv = 0

    conv_pos = [0]*100
    conv_len = [0]*100
    un_len= [0]*100
    
    align_path = '/home/allen/data/4suv/genome_aligned/riboseq_oct/'+sample+'/'
    bamfile = pysam.AlignmentFile(align_path+sample+"_Aligned.out.sorted.bam","rb")
    
    for read in bamfile.fetch():  

        has_conversion = False

        if read.query_length != 30:
            continue    

        if read.query_length > 80:
            continue

        totalreads += 1 

        if read.get_tag('NM') >= 1:

            if (('I' in read.cigarstring) or ('D' in read.cigarstring) or ('S' in read.cigarstring)):
                continue

            MD = read.get_tag('MD')

            if read.is_reverse & ('G' in MD):
                mismatches = [x.start() for x in re.finditer('g', read.get_reference_sequence())]
                for i in mismatches:
                    if read.query_alignment_sequence[i] == 'A':
                        pos = read.reference_start + 1 + i
                        if (read.reference_name + '_' + str(pos)) in bcf_df.pos.values:
                            minus_snp += 1
                        else:
                            conv += 1
                            conv_pos[read.query_alignment_length - i] += 1
                            has_conversion = True

            elif (not read.is_reverse) & ('C' in MD):
                mismatches = [x.start() for x in re.finditer('c', read.get_reference_sequence())]
                for i in mismatches:
                    if read.query_alignment_sequence[i] == 'T':
                        pos = read.reference_start + 1 + i
                        if (read.reference_name + '_' + str(pos)) in bcf_df.pos.values:
                            plus_snp += 1
                        else:
                            conv += 1
                            conv_pos[i] += 1
                            has_conversion = True

        if has_conversion:
            conv_len[read.query_length] += 1
        else:
            un_len[read.query_length] += 1                        

    print(sample)
    print(totalreads,'reads analyzed')
#     print(minus_snp, 'minus snp')
#     print(plus_snp, 'plus snp')
    print(conv, 'true conversions')
    
    return [sample, totalreads, conv_pos, conv_len, un_len]

In [4]:
filelist = [
 'RIBO_UNT_R1',
 'RIBO_UNT_R2',
 'RIBO_UNT_R3',
 'RIBO_4SU_R1',
 'RIBO_4SU_R2',
 'RIBO_4SU_R3',
 'RIBO_4SU_UV_R1',
 'RIBO_4SU_UV_R2',
 'RIBO_4SU_UV_R3',
 'RIBO_UV_R1',
 'RIBO_UV_R2',
 'RIBO_UV_R3'
]

In [5]:
outdict = {}

p = Pool(12)
map_out = p.map(get_other_stats, filelist)
p.close()

for entry in map_out:
    outdict[entry[0]] = entry[1:]
    

RIBO_4SU_UV_R2
301641 reads analyzed
1648 true conversions
RIBO_4SU_UV_R3
415386 reads analyzed
2429 true conversions
RIBO_4SU_R2
529468 reads analyzed
2807 true conversions
RIBO_4SU_R1
559209 reads analyzed
2996 true conversions
RIBO_UV_R1
677578 reads analyzed
3543 true conversions
RIBO_UNT_R1
769368 reads analyzed
4077 true conversions
RIBO_4SU_R3
728106 reads analyzed
3896 true conversions
RIBO_UV_R3
1039946 reads analyzed
5593 true conversions
RIBO_4SU_UV_R1
884042 reads analyzed
5713 true conversions
RIBO_UNT_R2
1096483 reads analyzed
6289 true conversions
RIBO_UV_R2
1269577 reads analyzed
6503 true conversions
RIBO_UNT_R3
1336752 reads analyzed
6517 true conversions


In [6]:
# percentage of reads with a conversion 
# == 100 * reads with conversions / total reads

for fn in filelist:
    print(fn, 100 * sum(outdict[fn][2]) / outdict[fn][0])

RIBO_UNT_R1 0.524586413783781
RIBO_UNT_R2 0.5659002465154499
RIBO_UNT_R3 0.4841586173052294
RIBO_4SU_R1 0.5300343878585645
RIBO_4SU_R2 0.5242998632589694
RIBO_4SU_R3 0.531516015525212
RIBO_4SU_UV_R1 0.6400148409238475
RIBO_4SU_UV_R2 0.5397144287414509
RIBO_4SU_UV_R3 0.5782573317348202
RIBO_UV_R1 0.5181691259161307
RIBO_UV_R2 0.5077281645776507
RIBO_UV_R3 0.533393080025309
